# Ceneo Scraper

## Załadowanie biblioteki

In [7]:
import requests
from bs4 import BeautifulSoup
import json 
import os

## Funkcja do 

In [8]:
def extract(ancestor, selector, attribute=None,return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return  ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

## Selektory składowych opini

In [9]:
selectors = {
            "opinion_id": (None, "data-entry-id"),
            "author":("span.user-post__author-name",),
            "recommendation": ("span.user-post__author-recomendation",),
            "star": ("span.user-post__score-count",),
            "content": ("div.user-post__text",),
            "pros": ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
            "cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
            "helpful": ("button.vote-yes > span",),
            "unhelpful": ("button.vote-no > span",),
            "publish_date": ('span.user-post__published > time:nth-child(1)',"datetime"),
            "purchase_date": ('span.user-post__published > time:nth-child(2)',"datetime"),
        
}

## Wysyłanie Zapytania do serwera

In [10]:
# product_id = "138331381"
product_id = "39562616"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"


## Pobranie wszystkich opinii 

In [11]:
all_opinions = []
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
              for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + page_dom.select_one("a.pagination_next")["href"].strip()
    except TypeError: url = None
    print(url)

None


## Ekstrakcja składowych pojedyńczej opinii
|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id] |opinion_id|
|autor|span.user-post_author-name|author|
|rekomendacje|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|star|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature_title--positives ~ div.review-feature_item|pros|
|lista wad|div.review-feature_title--negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna|button.vote-no > span|unhelpful|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|

## Zapis opinii o produkcie w JSON


In [12]:
if not os.path.exists("opinions"):
    os.makedirs("opinions")
with open(f"opinions/{product_id}.json","w",encoding="utf-8") as jf:
    json.dump(all_opinions, jf,indent = 4, ensure_ascii = False)